# J2_VOP benchmark 

<ul id="top">
<li><a href="#Loading-packages"> 
             Loading Packages</a></li>
    
<li><a href="#Initial-value-problem:-J2-problem-(in-physical-time)">
              Initial value problem: J2-problem (in physical time)</a></li>
    
<li><a href="#Initial-value-problem:-VOP-formulation-of-$J_2$-KS-equation">
              Initial value problem: VOP formulation of $J_2$-KS equation</a></li>

<li><a href="#Work-precision-diagrams">
             Work-precision diagrams</a></li>

<li><a href="#Work-precision-diagrams-II-(DiffEqDevTools)">
             Work-precision diagrams-II (DiffEqDevTools) </a></li>

</ul>  

## Loading packages and functions

In [1]:
using LinearAlgebra
using Plots
using DiffEqDevTools,BenchmarkTools
using OrdinaryDiffEq
using JLD2, FileIO
using Parameters,NBInclude
using IRKGaussLegendre

In [2]:
PATH_SRC="../../src_simd/"
include(string(PATH_SRC,"IRKGL_SIMD.jl"))
using .IRKGL_SIMD   


PATH_SRC="../../src_seq/"
include(string(PATH_SRC,"IRKGL_SEQ.jl"))
using .IRKGL_SEQ  


PATH_SRC="../../src/"
include(string(PATH_SRC,"MyBenchmarksTools.jl"))

launch_method_tests (generic function with 1 method)

In [3]:
run=false

false

##### <a href="#top">Hasierara</a>

## Initial value problem: J2-problem (in physical time)

In [9]:
#PATH_ODES="../../ODEProblems/"
#@nbinclude(string(PATH_ODES,"J2_Problem.ipynb"));

function Energy(u,p)
    μ = p[1]
    R = p[2]
    ϵ = p[3]
    q = u[1:3]
    v = u[4:6]
    r = norm(q)
    v2 = dot(v,v)
    sinth2 = (u[3]/r)^2
    aux = (R/r)^2
    return 0.5*v2 - μ/r * (1 + 0.5*ϵ*aux * (1 - 3*sinth2))
end

function J2ODE!(du,u,p,t)
    μ = p[1]
    R = p[2]
    ϵ = p[3]
    x = u[1]
    y = u[2]
    z = u[3]
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]
    r2 = x^2+y^2+z^2
    r = sqrt(r2)
    aux1 = -μ/(r*r2)
    F = 1.5 - 7.5*(z/r)^2
    G =  3 + F
    aux2 = ϵ*(R/r)^2
    aux3 = aux1*(1+aux2*F)
    aux4 = aux1*(1+aux2*G)
    du[4]=aux3*x
    du[5]=aux3*y
    du[6]=aux4*z
    return nothing
end

J2ODE! (generic function with 1 method)

In [10]:
#q0 = [0., 37947.73745727695, 0.]
#v0 = [3.297676220718193,0., 0.8244190551795483]

q0 = [0., 9486.934364319237, 0.]
v0 = [6.595352441436386, 0., 1.6488381103590966]

u0 = vcat(q0,v0)
dim=length(size(u0))

μ = 398600.8
Re = 6378.135
ϵ = 0.0010826157
p =  [μ, Re, ϵ]
E0 = Energy(u0,p)

-18.91737400006957

In [11]:
s=8

t0 = 0.
birak = 10000
tF = birak*10770.5
dt = (tF-t0)/(10*birak)

n = 1000
#n = 5

m = convert(Int64,ceil(abs(tF-t0)/(n*dt)))
m=100
n = convert(Int64,ceil(abs(tF-t0)/(m*dt))) # Number of macro-steps (Output is saved for n+1 time values)
dt = (tF-t0)/(n*m)
println("dt = $dt, n=$n, m=$m")

prob = ODEProblem(J2ODE!, u0, (t0,tF), p)

alg=IRKGL_simd(s=s, initial_interp=1, dim=dim, floatType=Float64, m=m, myoutputs=true)
sol0,iters0=solve(prob, alg, dt=dt, adaptive=false);

dt = 1077.05, n=1000, m=100


LoadError: "No matching function wrapper was found!"

In [7]:
alg1=IRKGL_Seq(s=s, initial_interp=1,  m=m, myoutputs=true)
asol1,iters1=solve(prob, alg1, dt=dt, adaptive=false);

In [ ]:
plot(sol0.t[2:end],  iters0[2:end], title="Iteration numbers", legend=false, size=(800,200))
plot!(sol1.t[2:end], iters1[2:end], legend=false)

In [ ]:
E0 = Energy(BigFloat.(u0),p)
yrange = (1e-20,1e-10)
year = 365.5
epsilon = eps(1e-3)
tt = sol0.t[2:end]/year
energia_erroreak0 = [Float64.(abs(Energy(BigFloat.(u),p)/E0-1)) for u in sol0.u[2:end]]
energia_erroreak1 = [Float64.(abs(Energy(BigFloat.(u),p)/E0-1)) for u in sol1.u[2:end]]


plot(title="Energia errore erlatiboen eboluzioa",
         yscale=:log10, ylims=yrange, legend=:bottomright, size=(800,200))
plot!(tt,  energia_erroreak0, label="IRK16_SIMD")
plot!(tt, energia_erroreak1, label="IRK16_Seq")



In [ ]:
xx = [u[1] for u in sol0.u]
yy = [u[2] for u in sol0.u]
plot(xx,yy, aspect_ratio=1, title="Satellite J2 Problem", legend=false, xlabel="x", ylabel="y")

##### <a href="#top">Back to the top</a>

## Initial value problem: VOP formulation of $J_2$-KS equation


In [ ]:
PATH_ODES="../../ODEProblems/"

@nbinclude(string(PATH_ODES,"J2_VOP.ipynb"));

In [ ]:
q0 = [0., 37947.73745727695, 0.]
v0 = [3.297676220718193,0., 0.8244190551795483]
μ = 398600.8
h = μ/norm(q0) - 0.5*dot(v0,v0) -  V(q0) 
ω = sqrt(h/2)

u0 = χ(q0)
V0 = vcat(v0,[0.])
w0 = 0.5*L(u0)' * V0
r0 = norm(q0)

dim=length(size(u0))

# Konprobazioa:
(norm(L(u0)*u0 - vcat(q0,[0.])), norm(r0*V0 - 2* L(u0) * w0))

In [ ]:
Ifcn(u0,w0)

### VOP formulation




In [ ]:
α0 = u0
β0 = w0
U0 = vcat(α0, β0)
μ = 398600.8
h = μ/norm(q0) - 0.5*dot(v0,v0) -  V(q0)
ω = sqrt(h/2)
C = 1.7554962315534863e10
p = [C, ω, μ];

### Integration parameters

In [ ]:
s=8

t0 = 0.
sasi_periodo = 2*π/ω
sasi_periodo = 6/ω
dt = sasi_periodo/5
tF = 20000*sasi_periodo
tF = 2000*sasi_periodo

n = 1000
#n = 5

m = convert(Int64,ceil(abs(tF-t0)/(n*dt)))
m=100
n = convert(Int64,ceil(abs(tF-t0)/(m*dt))) # Number of macro-steps (Output is saved for n+1 time values)
dt = (tF-t0)/(n*m)
println("dt = $dt, n=$n, m=$m")

prob = ODEProblem(J2VOPODE!, U0, (t0,tF), p)

tspan_B=(BigFloat(t0),BigFloat(tF))
U0_B=BigFloat.(U0)
p_B=BigFloat.(p)

prob_B = ODEProblem(J2VOPODE!, U0_B, tspan_B, p_B);

##### <a href="#top">Back to the top</a>

## Work-precision diagrams

### Test-Solution

In [ ]:
if run==true
   sol =solve(prob_B,Vern9(),save_everystep=false, abstol=1e-16,reltol=1e-16)
   @save "./Data/VOP_sol_T2000.jld2" sol
else
   @load "./Data/VOP_sol_T2000.jld2" sol
end
   
test_sol = TestSolution(sol)
final_state=sol.u[end]

E0=J2VOPEnergy(sol.u[1],p_B,sol.t[1])
(Float32(sol.t[end]),Float32(J2VOPEnergy(sol.u[end],p_B,sol.t[end])/E0-1))

### Integrations

In [ ]:
tols=abstols=reltols=[1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]

dts8= [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1., 1.05, 1.1]
dtsVern=dts8/2;

In [ ]:
nruns=100

s=8

wp1=launch_IRKGL16_tests(IRKGL16(),final_state, prob, dts8; adaptive=false, nruns=nruns)
wp2=launch_IRKGL_tests(IRKGL_simd, final_state, prob, s, dts8; dim=dim, adaptive=false,
                       initial_interp=0, nruns=nruns);

wp11=launch_method_tests(Vern9(), final_state, prob, dtsVern, adaptive=false, nruns=nruns)
wp12=launch_method_tests(Vern9(), final_state, prob, adaptive=true, tols, nruns=nruns);

### Plots: IRKGL16-seq vs IRKGL16-simd

In [ ]:
yrange=(10^(-2.5),10^(-1.))

plot(title="J2_VOP ",xlabel="Error", ylabel="Time (s)", ylims=yrange,
    xticks=[1e-16,1e-15,1e-14,1e-13,1e-12, 1e-11, 1e-10])
# 
plot!(wp1.errors, wp1.times, seriestype=:scatter, scale=:log10, label="",color="blue") #label="IRKGL16"
plot!(wp1.errors, wp1.times, scale=:log10, label="", lw=3, color="blue")

#
plot!(wp2.errors, wp2.times, seriestype=:scatter, scale=:log10, label="",color="green") #label="IRKGL16_simd"
plot!(wp2.errors, wp2.times, scale=:log10, label="", lw=3, color="green")
#
#

### Plots: IRKGL16-simd vs Vern9/DPRKN12

In [ ]:
plot(title="J2_VOP benchmark",xlabel="Error", ylabel="Time (s)")
# 
plot!(wp12.errors, wp12.times, seriestype=:scatter, scale=:log10, label="Vern9-Adap",color="red")
plot!(wp12.errors, wp12.times, scale=:log10, label="", lw=3, color="red")
#
plot!(wp11.errors, wp11.times, seriestype=:scatter, scale=:log10, label="Vern9-Fix",color="blue")
plot!(wp11.errors, wp11.times, scale=:log10, label="", lw=3, color="blue")

#
plot!(wp2.errors, wp2.times, seriestype=:scatter, scale=:log10, label="IRKGL_simd_s8",color="green")
plot!(wp2.errors, wp2.times, scale=:log10, label="", lw=3, color="green")
#
#

In [ ]:
yrange=(10^(-2.25),10^(-1.5))

plot(title="J2_VOP",xlabel="Error", ylabel="Time (s)", ylims=yrange,
     xticks=[1e-16,1e-15,1e-14,1e-13,1e-12,1e-11, 1e-10])
# 
#plot!(wp12.errors, wp12.times, seriestype=:scatter, scale=:log10, label="Vern9-Adap",color="red")
#plot!(wp12.errors, wp12.times, scale=:log10, label="", lw=2, color="red")
#
 plot!(wp11.errors, wp11.times, seriestype=:scatter, scale=:log10, label="",color="orange") #label="Vern9"
 plot!(wp11.errors, wp11.times, scale=:log10, label="", lw=3, color="orange")

#
plot!(wp2.errors, wp2.times, seriestype=:scatter, scale=:log10, label="",color="green") #label="IRKGL16_simd"
plot!(wp2.errors, wp2.times, scale=:log10, label="", lw=3, color="green")
#

##### <a href="#top">Back to the top</a>

## Work-precision diagrams-II (DiffEqDevTools)

In [ ]:
nruns=100
s=8

setups = [
           Dict(:alg=>Vern9(),:adaptive=>false,:dts=>dtsVern)
           Dict(:alg=>IRKGL_simd(s=s, dim=dim,initial_interp=0),:adaptive=>false,:dts=>dts8)
]
solnames = ["Vern9","IRKGL16-simd"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=nruns,names=solnames);

### plot(wp)

In [ ]:
plot(wp)